## Tools

Tools provide llms the capability to do tasks -LLms-Reasoning, Tools-Work<BR>

LangChain supports 
1. Inbuilt tools
2. Custom tools that can be made as per the requirement

## Inbuilt tools

Some of the tools that will be discussed in the notebook are
1. Search tool- Google search(paid), Brave search, Duck-Duck-go
2. Shell tool- allows to execute shell commands

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# installing the required libraries
!pip install -U ddgs

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool=DuckDuckGoSearchRun()
print(search_tool.invoke("What's the trending news in India today?"))

c:\Users\mudas\OneDrive\Desktop\Langchain\my_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


15 hours ago · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. 2 days ago · Headlines today: Get news headlines from India and around the world. Today's breaking news on politics, sports, entertainment, business, life style and many more on Times of India 1 day ago · News Today Live Updates, 26 August | Punjab orders closure of schools in several districts as flood crisis worsens Latest News Live Updates | Excess water release from dams has inundated large swathes of agricultural land across several districts. 1 day ago · Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusive news stories on Indian politics, current affairs, cricket matches, festivals and events 5 days ago · Trending news and viral stories from India and the world. Get

In [4]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Shell tool

In [6]:
from langchain_community.tools import ShellTool

shell=ShellTool()
print(shell.invoke("dir"))

Executing command:
 dir
 Volume in drive C has no label.
 Volume Serial Number is 5ED7-C4E7

 Directory of c:\Users\mudas\OneDrive\Desktop\Langchain\6_Tools

27-08-2025  02.37 PM    <DIR>          .
27-08-2025  02.36 PM    <DIR>          ..
27-08-2025  03.04 PM             6,103 Inbuilt_custom_tools.ipynb
               1 File(s)          6,103 bytes
               2 Dir(s)  256,069,025,792 bytes free



c:\Users\mudas\OneDrive\Desktop\Langchain\my_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Building Custom Tools

### Method 1 -> @tool

In [8]:
from langchain_core.tools import tool

In [10]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [11]:
# Step 2 - add type hints

def multiply(a:int , b:int)-> int :
    """Multiply two numbers"""
    return a*b

In [12]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [13]:
multiply.invoke({'a': 3, 'b': 4})

12

In [17]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.model_json_schema())

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method-2 -> Using structured tool

In [19]:
from pydantic import Field,BaseModel
from langchain.tools import StructuredTool

In [30]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a:int=Field(description="The first number to add",json_schema_extra={'required':True})
    b:int=Field(description="The second number to add",json_schema_extra={'required':True})

In [31]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [32]:
multiply_tool=StructuredTool(
    func=multiply_func,
    name="multiply",
    description="Multiplies two numbers",
    args_schema=MultiplyInput
)

In [33]:
multiply_tool.invoke({'a': 2, 'b': 3})

6

In [34]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiplies two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Method-3-> using BaseTool Class

In [35]:
from langchain.tools import BaseTool
from typing import Type

In [37]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add", json_schema_extra={'required': True})
    b: int = Field(description="The second number to add", json_schema_extra={'required': True})

In [38]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [39]:
tool=MultiplyTool()

In [40]:
result = tool.invoke({'a':3, 'b':3})

print(result)
print(tool.name)
print(tool.description)

print(tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Tookit

In [41]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [42]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [43]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
